# Introduction

Bölüm 1'de gördüğünüz gibi, Transformatör modelleri genellikle çok büyüktür. Milyonlarca ila on milyarlarca parametre içeren bu modelleri eğitmek ve dağıtmak karmaşık bir iştir. Ayrıca, neredeyse her gün yeni modellerin piyasaya sürülmesi ve her birinin kendi uygulamasına sahip olması nedeniyle, hepsini denemek kolay bir iş değildir.

Transformers kütüphanesi bu sorunu çözmek için oluşturulmuştur. Amacı, herhangi bir Transformer modelinin yüklenebileceği, eğitilebileceği ve kaydedilebileceği tek bir API sağlamaktır. Kütüphanenin ana özellikleri şunlardır:

- Kullanım kolaylığı: Son teknoloji ürünü bir NLP modelinin indirilmesi, yüklenmesi ve çıkarım için kullanılması sadece iki satır kodla yapılabilir. 

- Esneklik: Özünde, tüm modeller basit PyTorch nn.Module veya TensorFlow tf.keras.Model sınıflarıdır ve ilgili makine öğrenimi (ML) çerçevelerindeki diğer modeller gibi ele alınabilir. 

- Basitlik: Kütüphane genelinde neredeyse hiç soyutlama yapılmamıştır. "Hepsi tek bir dosyada" temel bir kavramdır: bir modelin ileri geçişi tamamen tek bir dosyada tanımlanır, böylece kodun kendisi anlaşılabilir ve hacklenebilir. 

Bu son özellik Transformers'ı diğer ML kütüphanelerinden oldukça farklı kılar. Modeller, dosyalar arasında paylaşılan modüller üzerine inşa edilmemiştir; bunun yerine, her modelin kendi katmanları vardır. Bu, modelleri daha ulaşılabilir ve anlaşılabilir kılmanın yanı sıra, bir model üzerinde diğerlerini etkilemeden kolayca deney yapabilmenizi sağlar.

Bu bölüm, Bölüm 1'de tanıtılan pipeline() fonksiyonunu kopyalamak için bir model ve bir tokenizer'ı birlikte kullandığımız uçtan uca bir örnekle baş""layacaktır. Ardından, model API'sini tartışacağız: model ve yapılandırma sınıflarına dalacağız ve bir modelin nasıl yükleneceğini ve tahminlerin çıktısını almak için sayısal girdileri nasıl işlediğini göstereceğiz.

Ardından pipeline() fonksiyonunun diğer ana bileşeni olan tokenizer API'sine bakacağız. Tokenizer'lar, sinir ağı için metinden sayısal girdilere dönüştürme ve gerektiğinde metne geri dönüştürme işlemlerini gerçekleştirerek ilk ve son işleme adımlarıyla ilgilenir. Son olarak, birden fazla cümlenin bir model aracılığıyla hazırlanmış bir yığın halinde nasıl gönderileceğini göstereceğiz ve ardından üst düzey tokenizer() işlevine daha yakından bakarak her şeyi tamamlayacağız.

# Behind the pipeline

Bölüm 1'de aşağıdaki kodu çalıştırdığımızda perde arkasında neler olduğuna bir göz atarak tam bir örnekle başlayalım:

In [1]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

2024-08-01 19:11:42.230271: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 19:11:42.230460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 19:11:42.441068: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

Bölüm 1'de gördüğümüz gibi, bu pipeline üç adımı bir araya getirir: ön işleme, girdileri modelden geçirme ve son işleme:

![image1](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline-dark.svg)

## Preprocessing with a tokenizer

Diğer sinir ağları gibi Transformer modelleri de ham metni doğrudan işleyemez, bu nedenle boru hattımızın ilk adımı metin girdilerini modelin anlamlandırabileceği sayılara dönüştürmektir. Bunu yapmak için aşağıdakilerden sorumlu olacak bir tokenizer kullanıyoruz:

- Girdiyi token adı verilen kelimelere, alt kelimelere veya sembollere (noktalama işaretleri gibi) bölme
- Her bir tokeni bir tamsayıya eşleme
- Modele faydalı olabilecek ek girdilerin eklenmesi


Tüm bu ön işlemlerin modelin ön eğitime tabi tutulmasıyla tamamen aynı şekilde yapılması gerekir, bu nedenle öncelikle bu bilgileri Model Hub'ından indirmemiz gerekir. Bunu yapmak için AutoTokenizer sınıfını ve onun from_pretrained() yöntemini kullanırız. Modelimizin kontrol noktası adını kullanarak, modelin tokenizer'ıyla ilişkili verileri otomatik olarak getirecek ve önbelleğe alacaktır (böylece yalnızca aşağıdaki kodu ilk kez çalıştırdığınızda indirilir).

Sentiment-analysis pipeline'ın varsayılan kontrol noktası distilbert-base-uncased-finetuned-sst-2-english olduğundan (model kartını burada görebilirsiniz), aşağıdakileri çalıştırıyoruz:

In [1]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Bir kez tokenizer'a sahip olduğumuzda, cümlelerimizi doğrudan ona aktarabiliriz ve modelimize beslemeye hazır bir sözlük geri alırız! Geriye kalan tek şey, input ID'lerinin listesini tensörlere dönüştürmektir.

Arka uç olarak hangi ML çerçevesinin kullanıldığı konusunda endişelenmenize gerek kalmadan  Transformers'ı kullanabilirsiniz; PyTorch veya TensorFlow veya bazı modeller için Flax olabilir. Ancak, Transformer modelleri girdi olarak yalnızca tensörleri kabul eder. Tensörleri ilk kez duyuyorsanız, bunları NumPy dizileri olarak düşünebilirsiniz. Bir NumPy dizisi skaler (0D), vektör (1D), matris (2D) veya daha fazla boyuta sahip olabilir. Etkili bir şekilde bir tensördür; diğer ML çerçevelerinin tensörleri benzer şekilde davranır ve genellikle NumPy dizileri gibi örneklenmesi kolaydır.

Geri almak istediğimiz tensör türünü (PyTorch, TensorFlow veya düz NumPy) belirtmek için return_tensors argümanını kullanırız:

In [2]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


Henüz dolgu ve kesme konusunda endişelenmeyin; bunları daha sonra açıklayacağız. Burada hatırlanması gereken ana şeyler, bir cümle veya bir cümle listesi geçebileceğiniz ve geri almak istediğiniz tensörlerin türünü belirtebileceğinizdir (herhangi bir tür geçilmezse, sonuç olarak bir liste listesi alırsınız).

İşte sonuçların PyTorch tensörleri olarak nasıl göründüğü:

Çıktının kendisi iki anahtar içeren bir sözlüktür, input_ids ve attention_mask. input_ids, her cümledeki belirteçlerin benzersiz tanımlayıcıları olan iki tamsayı satırı (her cümle için bir tane) içerir. attention_mask'ın ne olduğunu bu bölümün ilerleyen kısımlarında açıklayacağız.

## Going through the model

Önceden eğitilmiş modelimizi tokenizer ile yaptığımız gibi indirebiliriz. Transformers, aynı zamanda bir **from_pretrained()** yöntemine sahip olan bir **AutoModel** sınıfı sağlar:

In [3]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Bu kod parçacığında, daha önce pipeline'ımızda kullandığımız aynı kontrol noktasını indirdik (aslında zaten önbelleğe alınmış olmalıydı) ve onunla bir model oluşturduk.

Bu mimari yalnızca temel Transformer modülünü içerir: bazı girdiler verildiğinde, özellikler olarak da bilinen gizli durumlar olarak adlandıracağımız çıktıları verir. Her model girdisi için, Transformer modeli tarafından bu girdinin bağlamsal olarak anlaşılmasını temsil eden yüksek boyutlu bir vektör alacağız.

Bu bir anlam ifade etmiyorsa endişelenmeyin. Hepsini daha sonra açıklayacağız.

Bu gizli durumlar kendi başlarına faydalı olsalar da, genellikle modelin kafa olarak bilinen başka bir bölümüne girdilerdir. Bölüm 1'de, farklı görevler aynı mimariyle gerçekleştirilebilirdi, ancak bu görevlerin her biri kendisiyle ilişkili farklı bir kafaya sahip olacaktır.

Yüksek boyutlu bir vektör mü?

Transformatör modülünün vektör çıktısı genellikle büyüktür. Genellikle üç boyuta sahiptir:

- Batch size: Bir seferde işlenen dizi sayısı (örneğimizde 2).
- Dizi uzunluğu: Dizinin sayısal temsilinin uzunluğu (örneğimizde 16).
- Gizli boyut: Her bir model girdisinin vektör boyutu.

Son değer nedeniyle "yüksek boyutlu" olduğu söylenir. Gizli boyut çok büyük olabilir (daha küçük modeller için 768 yaygındır ve daha büyük modellerde bu 3072 veya daha fazlasına ulaşabilir).

Ön işlemden geçirdiğimiz girdileri modelimize beslersek bunu görebiliriz:

In [4]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


Transformers modellerinin çıktılarının **namedtuple**s veya sözlükler gibi davrandığını unutmayın. Öğelere niteliklere göre (bizim yaptığımız gibi) veya anahtara göre (outputs["last_hidden_state"]), hatta aradığınız şeyin tam olarak nerede olduğunu biliyorsanız dizine göre (outputs[0]) erişebilirsiniz.

## Model heads: Making sense out of numbers

Model kafaları, yüksek boyutlu gizli durum vektörünü girdi olarak alır ve bunları farklı bir boyuta yansıtır. Genellikle bir veya birkaç doğrusal katmandan oluşurlar:

![image9](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head-dark.svg)

Transformatör modelinin çıktısı işlenmek üzere doğrudan model kafasına gönderilir.

Bu diyagramda model, gömme katmanı ve sonraki katmanlar tarafından temsil edilmektedir. Gömme katmanı, tokenize edilmiş girdideki her bir girdi kimliğini ilişkili tokenı temsil eden bir vektöre dönüştürür. Sonraki katmanlar, cümlelerin nihai temsilini üretmek için dikkat mekanizmasını kullanarak bu vektörleri manipüle eder.

Transformers'da birçok farklı mimari mevcuttur ve her biri belirli bir görevin üstesinden gelmek üzere tasarlanmıştır. İşte kapsamlı olmayan bir liste:

- *Model (retrieve the hidden states)
- *ForCausalLM
- *ForMaskedLM
- *ForMultipleChoice
- *ForQuestionAnswering
- *ForSequenceClassification
- *ForTokenClassification
- and others

Örneğimiz için, dizi sınıflandırma başlığına sahip bir modele ihtiyacımız olacak (cümleleri pozitif veya negatif olarak sınıflandırabilmek için). Bu yüzden, aslında AutoModel sınıfını değil, AutoModelForSequenceClassification sınıfını kullanacağız:

In [5]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

Şimdi çıktılarımızın şekline bakarsak, boyutluluk çok daha düşük olacaktır: model kafası, daha önce gördüğümüz yüksek boyutlu vektörleri girdi olarak alır ve iki değer içeren vektörleri (etiket başına bir tane) çıktı olarak verir:

In [6]:
print(outputs.logits.shape)

torch.Size([2, 2])


Elimizde sadece iki cümle ve iki etiket olduğu için modelimizden elde ettiğimiz sonuç 2 x 2 şeklindedir.

## Postprocessing the output

Modelimizden çıktı olarak elde ettiğimiz değerler kendi başlarına bir anlam ifade etmeyebilir. Bir göz atalım:

In [7]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


Modelimiz ilk cümle için [-1.5607, 1.6123] ve ikinci cümle için [ 4.1692, -3.3464] tahmininde bulunmuştur. Bunlar olasılık değil logittir, modelin son katmanı tarafından çıkarılan ham, normalize edilmemiş puanlardır. Olasılıklara dönüştürülmeleri için bir SoftMax katmanından geçmeleri gerekir (tüm 🤗 Transformers modelleri logit çıktısı verir, çünkü eğitim için kayıp fonksiyonu genellikle SoftMax gibi son aktivasyon fonksiyonunu çapraz entropi gibi gerçek kayıp fonksiyonu ile birleştirir):

In [8]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


Şimdi modelin ilk cümle için [0.0402, 0.9598] ve ikinci cümle için [0.9995, 0.0005] tahmininde bulunduğunu görebiliriz. Bunlar tanınabilir olasılık puanlarıdır.

Her bir pozisyona karşılık gelen etiketleri elde etmek için model yapılandırmasının id2label niteliğini inceleyebiliriz (bu konuda daha fazla bilgi bir sonraki bölümde):

In [9]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

Şimdi modelin aşağıdakileri öngördüğü sonucuna varabiliriz:

- İlk cümle: NEGATİF: 0,0402, POZİTİF: 0,9598 
- İkinci cümle: NEGATİF: 0.9995, POZİTİF: 0.0005 

Pipeline'nın üç adımını başarıyla yeniden ürettik: belirteçlerle ön işleme, girdileri aktarma